In [1]:
import netpixi
from graph_tool import draw
import graph_tool_extras as gte

import time

In [2]:
def get_or_add_vertex(g, id):
    v = g.vertex_by_id(id)
    if v is None:
        v = g.add_vertex_by_id(id)
    return v

def get_or_add_edge(g, a, b, v):
    e = g.edge_by_ids(a, b)
    if e is None:
        e = g.add_edge_by_ids(a, b)
        e['orders'] = v
    return e

In [3]:
g = gte.Graph(directed=False)
g.add_ep('orders')

with open('./data/edges.csv') as file:
    for line in file:
        a, b, v = [int(x) for x in line.strip().replace('\n', '').split(',')]
        vA = get_or_add_vertex(g, a)
        vB = get_or_add_vertex(g, b)
    
        e = get_or_add_edge(g, a, b, v)

### 04: Construindo e renderizando a Rede

In [4]:
layout = draw.sfdp_layout(g)

In [5]:
gte.move(g, layout)

In [6]:
g = gte.clean(g)

In [7]:
gte.save(g, 'rede.net.gz')

In [8]:
r = netpixi.render('./rede.net.gz', infinite=True)

In [9]:
time.sleep(5)
r.vertex_default(size=4, bwidth=1)
r.edge_default(width=1)